In [ ]:
import argparse
import os
from collections import defaultdict
from itertools import combinations, groupby

import numpy as np
import pandas as pd
import yaml

from config import SPECS_COMPLETE_YAMLS, SP_RESULTS_DIR, RO_RESULTS_DIR
from model import DC, LPAC, Solution
from util import in_notebook

In [ ]:
# user inputs
if in_notebook():
    f_min = 0
    f_max = 3
    f_inc = 1
    r_hat = 3
    casestudy = 'imelda'
    pftype_a = 'dc'
    pftype_b = 'lpacc'
    approach = 'robust'
    timelimit = 45 * 60
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--fmin')
    parser.add_argument('--fmax')
    parser.add_argument('--finc')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype-a')
    parser.add_argument('--pftype-b')
    parser.add_argument('--approach')
    parser.add_argument('--timelimit')
    args = parser.parse_args()
    f_min = int(args.fmin)
    f_max = int(args.fmax)
    f_inc = int(args.finc)
    r_hat = int(args.rhat)
    pftype_a = str(args.pftype_a)
    pftype_b = str(args.pftype_b)
    casestudy = str(args.casestudy)
    approach = str(args.approach)
    timelimit = int(args.timelimit)

In [ ]:
if approach == 'stochastic':
    MY_RESULTS_DIR = SP_RESULTS_DIR
elif approach == 'robust':
    MY_RESULTS_DIR = RO_RESULTS_DIR
else:
    raise ValueError('`approach` must be either "stochastic" or "robust"')

In [ ]:
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype_a]) as fh:
    specs_a = yaml.load(fh, Loader=yaml.Loader)
    specs_a['options']['approach'] = approach
    for key, val in specs_a['r_hat'].items():
        specs_a['r_hat'][key] = min(val, r_hat)
    for key in list(specs_a['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs_a['xi'].pop(key)
    for k in specs_a['R']:
        specs_a['R'][k] = [i for i in range(1, min(max(specs_a['R'][k]), r_hat) + 1)]

with open(SPECS_COMPLETE_YAMLS[casestudy, pftype_b]) as fh:
    specs_b = yaml.load(fh, Loader=yaml.Loader)
    specs_b['options']['approach'] = approach
    for key, val in specs_b['r_hat'].items():
        specs_b['r_hat'][key] = min(val, r_hat)
    for key in list(specs_b['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs_b['xi'].pop(key)
    for k in specs_b['R']:
        specs_b['R'][k] = [i for i in range(1, min(max(specs_b['R'][k]), r_hat) + 1)]

c = pd.Series(specs_a['c'])

In [ ]:
model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

In [ ]:
# instantiate models
MODEL_a = model_classes[pftype_a](**specs_a)
for (k, r) in MODEL_a.x:
    MODEL_a.x[k, r].lb = 0
    MODEL_a.x[k, r].ub = 0
MODEL_a.update()
MODEL_a.model.setParam('OutputFlag', 0)
MODEL_a.model.setParam('Threads', 4)
MODEL_a.model.setParam('Presolve', 2)
MODEL_a.model.setParam('Method', 2)
MODEL_a.model.setParam('IntFeasTol', 1e-8)

MODEL_b = model_classes[pftype_b](**specs_b)
for (k, r) in MODEL_b.x:
    MODEL_b.x[k, r].lb = 0
    MODEL_b.x[k, r].ub = 0
MODEL_b.update()
MODEL_b.model.setParam('OutputFlag', 0)
MODEL_b.model.setParam('Threads', 4)
MODEL_b.model.setParam('Presolve', 2)
MODEL_b.model.setParam('Method', 2)
MODEL_b.model.setParam('IntFeasTol', 1e-8)

In [ ]:
for f in np.arange(f_min, f_max + f_inc, f_inc):

    zipfile_a = os.path.join(MY_RESULTS_DIR, f'{casestudy}-{pftype_a}-f{f}-r{r_hat}.zip')
    sol_a = Solution.from_zip(zipfile_a)

    zipfile_b = os.path.join(MY_RESULTS_DIR, f'{casestudy}-{pftype_b}-f{f}-r{r_hat}.zip')
    sol_b = Solution.from_zip(zipfile_b)

    if (sol_a['x'].round() != sol_b['x'].round()).any():
        print(casestudy, approach, f, pftype_a, pftype_b,
              sol_a['x'].multiply(sol_b['x']).multiply(c).sum(),
              sol_a['x'].multiply(sol_b['x']).multiply(c).sum() / f, end=' ', flush=True)

        MODEL_a.con_resource_hi.RHS = f
        for (k, r), indicator in sol_b['x'].round().items():
            if indicator > 0.5:
                MODEL_a.x[k, r].lb = 1
                MODEL_a.x[k, r].ub = 1
        MODEL_a.update()
        MODEL_a.solve()
        for (k, r), indicator in sol_b['x'].round().items():
            if indicator > 0.5:
                MODEL_a.x[k, r].lb = 0
                MODEL_a.x[k, r].ub = 0
        obj_a = MODEL_a.model.getObjective().getValue()

        MODEL_b.con_resource_hi.RHS = f
        for (k, r), indicator in sol_a['x'].round().items():
            if indicator > 0.5:
                MODEL_b.x[k, r].lb = 1
                MODEL_b.x[k, r].ub = 1
        MODEL_b.update()
        MODEL_b.solve()
        for (k, r), indicator in sol_a['x'].round().items():
            if indicator > 0.5:
                MODEL_b.x[k, r].lb = 0
                MODEL_b.x[k, r].ub = 0
        obj_b = MODEL_b.model.getObjective().getValue()

        print((obj_a - sol_a['ObjVal']) / sol_a['ObjVal'] * 100,
              (obj_b - sol_b['ObjVal']) / sol_b['ObjVal'] * 100)